## M6 database coding session

In [ ]:
import pandas as pd
import numpy as np

In [4]:
# Not stable URL from here: https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/
url = "https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/_w_e57f2079/session/8211590d61fa5888eafd6dfb23839726/download/downloadData?w=e57f2079"

In [5]:
nba = pd.read_csv(url)
nba.head()

<ipython-input-5-e68c0c818c74>:1: DtypeWarning: Columns (68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  nba = pd.read_csv(url)


,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.049896,16.76,37.403623,20.176010,69.0,28.0,4.0,0.0,0.0,49.812593
1,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.126850,22.64,39.205128,26.506944,1.0,50.0,48.0,1.0,0.0,48.305206
2,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.138090,9.76,31.130769,20.932051,0.0,0.0,18.0,79.0,3.0,42.877343
3,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.143369,10.76,19.821212,10.313636,0.0,0.0,0.0,27.0,73.0,47.177900
4,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.000000,9.76,37.091667,12.512121,0.0,12.0,66.0,21.0,0.0,44.292150


In [12]:
# Set pd options display all 7:20 pm
pd.options.display.max_rows = None
nba.T

,0,1,2,3,4,5,6,7,8,9,...,115709,115710,115711,115712,115713,115714,115715,115716,115717,115718
game_id,202203130ATL,202203130ATL,202203130ATL,202203130ATL,202203130ATL,202203130ATL,202203130ATL,202203130ATL,202203130ATL,202203130ATL,...,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW
game_date,2022-03-13,2022-03-13,2022-03-13,2022-03-13,2022-03-13,2022-03-13,2022-03-13,2022-03-13,2022-03-13,2022-03-13,...,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07
OT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H_A,A,A,A,A,A,A,A,A,A,A,...,H,H,H,H,H,H,H,H,H,H
Team_Abbrev,IND,IND,IND,IND,IND,IND,IND,IND,IND,IND,...,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW
Team_Score,128,128,128,128,128,128,128,128,128,128,...,118,118,118,118,118,118,118,118,118,118
Team_pace,96.3,96.3,96.3,96.3,96.3,96.3,96.3,96.3,96.3,96.3,...,90.9,90.9,90.9,90.9,90.9,90.9,90.9,90.9,90.9,90.9
Team_efg_pct,0.601,0.601,0.601,0.601,0.601,0.601,0.601,0.601,0.601,0.601,...,0.606,0.606,0.606,0.606,0.606,0.606,0.606,0.606,0.606,0.606
Team_tov_pct,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
Team_orb_pct,35.7,35.7,35.7,35.7,35.7,35.7,35.7,35.7,35.7,35.7,...,18.9,18.9,18.9,18.9,18.9,18.9,18.9,18.9,18.9,18.9


In [13]:
nba = nba.drop('Inactives', axis=1)

#### TODOS: trategy for normalization down to the 3d NF:**

- T1: info about just the team
- T2: info about the team in each game 
- T3: info about the player in each game
- T4: inf about players, not dependent on games (season totals)
- T5: info about the teams, not dependent on games (season totals)

#### Information overall about the game

In [30]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 KB 8.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.3-cp38-cp38-macosx_10_9_x86_64.whl size=142673 sha256=1524b13def4da86bd28e3d42306c238d7b2a91550d443c5218a7f8e0c47a2d5c
  Stored in directory: /Users/dmitrymikhaylov/Library/Caches/pip/wheels/f3/dc/e2/b8e0e2142eff7fd680295ecd2d92e3bfbb90195523e43da161
Successfully built psycopg2
You should consider upgrading via the '/Users/dmitrymikhaylov/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [31]:
import dotenv
import sqlite3
import os
import psycopg2
from sqlalchemy import create_engine

In [14]:
nba.head(5)

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.049896,16.76,37.403623,20.176010,69.0,28.0,4.0,0.0,0.0,49.812593
1,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.126850,22.64,39.205128,26.506944,1.0,50.0,48.0,1.0,0.0,48.305206
2,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.138090,9.76,31.130769,20.932051,0.0,0.0,18.0,79.0,3.0,42.877343
3,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.143369,10.76,19.821212,10.313636,0.0,0.0,0.0,27.0,73.0,47.177900
4,202203130ATL,2022-03-13,0,A,IND,128,96.3,0.601,13.0,35.7,...,0.000000,9.76,37.091667,12.512121,0.0,12.0,66.0,21.0,0.0,44.292150


## Information overall about the game: OT, date, location, etc

In [17]:
nba2022 = nba.query("season==2022")

## Information overall about the game: OT, date, location, etc

In [19]:
game_info = nba2022[['game_id', 'game_date', 'OT']].drop_duplicates()
game_info.head()

,game_id,game_date,OT
0,202203130ATL,2022-03-13,0
24,202203130BOS,2022-03-13,0
50,202203130BRK,2022-03-13,0
73,202203130DET,2022-03-13,0
96,202203130NOP,2022-03-13,0


## Info about how the team overall did in the game

In [40]:
nba2022['win'] = nba2022['Team_Score'] > nba2022['Opponent_Score']
team_game = nba2022[['Team_Abbrev', 'H_A', 'win', 'game_id', 'fg', 'fga', 'fg3', 'fg3a', 
             'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']]
team_game = team_game.groupby(['game_id','Team_Abbrev']).agg({'H_A': pd.Series.mode,
                                                              'win': 'mean',
                                                              'fg': sum,
                                                              'fga': sum, 
                                                              'fg3': sum, 
                                                              'fg3a': sum, 
                                                              'ft': sum, 
                                                              'fta': sum, 
                                                              'orb': sum, 
                                                              'drb': sum, 
                                                              'ast': sum, 
                                                              'stl': sum, 
                                                              'blk': sum, 
                                                              'tov': sum, 
                                                              'pf': sum, 
                                                              'pts': sum})
team_game = team_game.reset_index()

<ipython-input-40-64475f1533d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba2022['win'] = nba2022['Team_Score'] > nba2022['Opponent_Score']


## Info about how the player did personally in the game

In [33]:
player_game = nba2022[['game_id', 'player_id', 'starter', 'minutes', 'fg', 'fga', 'fg3', 'fg3a', 
                   'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'usg_pct', 
                   'is_inactive', 'PG%', 'SG%', 'SF%', 'PF%', 'C%']]

player_game.head()

,game_id,player_id,starter,minutes,fg,fga,fg3,fg3a,ft,fta,...,tov,pf,pts,usg_pct,is_inactive,PG%,SG%,SF%,PF%,C%
0,202203130ATL,halibty01,1,40.083333,9,15,3,5,4,4,...,5,2,25,22.6,0,69.0,28.0,4.0,0.0,0.0
1,202203130ATL,hieldbu01,1,39.416667,9,20,2,7,5,6,...,2,5,25,26.0,0,1.0,50.0,48.0,1.0,0.0
2,202203130ATL,brissos01,1,28.966667,5,8,3,5,2,4,...,2,4,15,16.9,0,0.0,0.0,18.0,79.0,3.0
3,202203130ATL,jacksis01,1,27.900000,4,9,1,1,3,4,...,2,4,12,19.0,0,0.0,0.0,0.0,27.0,73.0
4,202203130ATL,taylote01,1,24.766667,6,8,1,2,3,4,...,0,0,16,16.4,0,0.0,12.0,66.0,21.0,0.0


## Info about the team's total season stats so far

In [28]:
nba2022.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'player',
       'player_id', 'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'plus_minus', 'did_not_play', 'is_inactive',
       'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct',
       'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct',
       'usg_pct', 'off_rtg', 'def_rtg', 'bpm', 'season', 'minutes',
       'double_double', 'triple_double', 'DKP', 'FDP', 'SDP', 'DKP_per_minute',
       'FDP_per_minute', 'SDP_per_minute', 'pf_per_minute', 'ts',
       'last_60_minutes_per_game_starting', 'la

In [35]:
teams = nba2022[['Team_Abbrev', 'game_id', 'fg', 'fga', 'fg3', 'fg3a', 
             'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']]
teams = teams.groupby('Team_Abbrev').agg({'game_id': lambda x: x.nunique(),
                                          'fg': sum,
                                          'fga': sum, 
                                          'fg3': sum, 
                                          'fg3a': sum, 
                                          'ft': sum, 
                                          'fta': sum, 
                                          'orb': sum, 
                                          'drb': sum, 
                                          'ast': sum, 
                                          'stl': sum, 
                                          'blk': sum, 
                                          'tov': sum, 
                                          'pf': sum, 
                                          'pts': sum})
WL = nba2022.groupby(['Team_Abbrev', 'game_id']).agg({'win': 'mean'})
WL = WL.groupby('Team_Abbrev').agg({'win':sum})
teams = pd.merge(teams, WL, on=['Team_Abbrev'], validate='one_to_one')

teams = teams.rename({'game_id':'total_games'}, axis=1)

teams = teams.reset_index()
teams

,Team_Abbrev,total_games,fg,fga,fg3,fg3a,ft,fta,orb,drb,ast,stl,blk,tov,pf,pts,win
0,ATL,67,2762,5884,841,2257,1193,1479,665,2282,1629,456,293,766,1246,7558,33.0
1,BOS,69,2752,6037,864,2520,1186,1455,748,2448,1654,498,416,909,1278,7554,41.0
2,BRK,68,2825,6021,768,2168,1153,1445,694,2308,1697,479,363,885,1365,7571,35.0
3,CHI,67,2820,5833,730,1948,1153,1414,577,2302,1616,481,293,821,1258,7523,41.0
4,CHO,68,2931,6349,954,2656,1101,1497,762,2357,1891,604,334,893,1384,7917,33.0
5,CLE,67,2640,5665,768,2201,1099,1441,684,2303,1680,477,288,917,1151,7147,38.0
6,DAL,68,2693,5909,885,2564,1087,1422,646,2319,1603,484,286,812,1370,7358,42.0
7,DEN,68,2794,5867,866,2470,1101,1394,616,2387,1859,496,257,937,1338,7555,40.0
8,DET,68,2566,6026,753,2328,1158,1485,760,2170,1557,509,323,926,1488,7043,18.0
9,GSW,68,2768,5901,987,2717,1050,1377,663,2430,1863,627,322,977,1404,7573,46.0


## Info about the player's total season so far

In [24]:
players = nba2022[['player', 'player_id', 'starter', 'minutes', 'fg', 'fga', 'fg3', 'fg3a', 
                   'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'usg_pct', 
                   'did_not_play', 'is_inactive', 'ts_pct', 'PG%', 'SG%', 'SF%', 'PF%', 'C%']]
players = players.groupby(['player','player_id']).agg({'starter': sum,
                                                   'minutes': 'mean',
                                                   'fg': sum,
                                                   'fga': sum, 
                                                   'fg3': sum, 
                                                   'fg3a': sum, 
                                                   'ft': sum, 
                                                   'fta': sum, 
                                                   'orb': sum, 
                                                   'drb': sum, 
                                                   'ast': sum, 
                                                   'stl': sum, 
                                                   'blk': sum, 
                                                   'tov': sum, 
                                                   'pf': sum, 
                                                   'pts': sum,
                                                  'usg_pct': 'mean',
                                                   'ts_pct': 'mean',
                                                  'did_not_play': sum,
                                                  'is_inactive': sum,
                                                  'PG%': 'mean', 
                                                   'SG%': 'mean', 
                                                   'SF%': 'mean', 
                                                   'PF%': 'mean', 
                                                   'C%': 'mean'}).add_prefix('season_')

In [36]:
players = players.reset_index()

In [37]:
players.head()

,player,player_id,season_starter,season_minutes,season_fg,season_fga,season_fg3,season_fg3a,season_ft,season_fta,...,season_pts,season_usg_pct,season_ts_pct,season_did_not_play,season_is_inactive,season_PG%,season_SG%,season_SF%,season_PF%,season_C%
0,Aaron Gordon,gordoaa01,62,31.671237,344,674,68,217,140,190,...,896,19.540323,0.594097,0,0,0.000000,0.000000,50.0,47.0,3.0
1,Aaron Henry,henryaa01,0,0.999020,1,5,0,1,0,0,...,2,6.835294,0.058824,11,0,0.000000,11.000000,69.0,21.0,0.0
2,Aaron Holiday,holidaa01,14,14.848305,136,289,34,88,46,54,...,352,17.145763,0.519102,6,0,99.186441,0.813559,0.0,0.0,0.0
3,Aaron Nesmith,nesmiaa01,2,8.159770,56,153,23,95,16,19,...,151,13.751724,0.268259,13,0,1.000000,30.000000,62.0,8.0,0.0
4,Aaron Wiggins,wiggiaa01,27,19.967708,119,249,35,106,39,55,...,312,11.118750,0.460875,7,2,7.000000,67.000000,25.0,1.0,0.0


## Create SQLite database

In [38]:
nbadb = sqlite3.connect("nba.dn")

In [41]:
game_info.to_sql('game_info', nbadb, index=False, chunksize=1000, if_exists = 'replace') # to be run

1018

In [42]:
team_game.to_sql('team_game', nbadb, index=False, chunksize=1000, if_exists = 'replace')

2036

In [43]:
player_game.to_sql('player_game', nbadb, index=False, chunksize=1000, if_exists = 'replace')

26220

In [44]:
teams.to_sql('teams_season', nbadb, index=False, chunksize=1000, if_exists = 'replace')

30

In [45]:
players.to_sql('players_season', nbadb, index=False, chunksize=1000, if_exists = 'replace')

612

In [48]:
pd.read_sql_query("SELECT * FROM players_season", nbadb).head(10)

,player,player_id,season_starter,season_minutes,season_fg,season_fga,season_fg3,season_fg3a,season_ft,season_fta,...,season_pts,season_usg_pct,season_ts_pct,season_did_not_play,season_is_inactive,season_PG%,season_SG%,season_SF%,season_PF%,season_C%
0,Aaron Gordon,gordoaa01,62,31.671237,344,674,68,217,140,190,...,896,19.540323,0.594097,0,0,0.000000,0.000000,50.0,47.0,3.0
1,Aaron Henry,henryaa01,0,0.999020,1,5,0,1,0,0,...,2,6.835294,0.058824,11,0,0.000000,11.000000,69.0,21.0,0.0
2,Aaron Holiday,holidaa01,14,14.848305,136,289,34,88,46,54,...,352,17.145763,0.519102,6,0,99.186441,0.813559,0.0,0.0,0.0
3,Aaron Nesmith,nesmiaa01,2,8.159770,56,153,23,95,16,19,...,151,13.751724,0.268259,13,0,1.000000,30.000000,62.0,8.0,0.0
4,Aaron Wiggins,wiggiaa01,27,19.967708,119,249,35,106,39,55,...,312,11.118750,0.460875,7,2,7.000000,67.000000,25.0,1.0,0.0
5,Abdel Nader,naderab01,0,6.050000,12,35,4,14,6,10,...,34,9.062500,0.200542,10,8,0.000000,13.000000,86.0,1.0,0.0
6,Ade Murkey,murkead01,0,1.466667,0,0,0,0,0,0,...,0,27.100000,0.568000,0,0,0.000000,0.000000,100.0,0.0,0.0
7,Admiral Schofield,schofad01,1,7.846491,36,79,14,45,7,10,...,93,10.389474,0.329447,8,0,0.000000,6.000000,54.0,39.0,1.0
8,Ahmad Caver,caverah01,0,0.208333,1,1,0,0,0,0,...,2,13.175000,0.250000,3,0,100.000000,0.000000,0.0,0.0,0.0
9,Al Horford,horfoal01,59,28.522500,230,506,75,237,73,86,...,608,15.275000,0.554717,1,0,0.000000,0.000000,0.0,1.0,99.0


In [49]:
pd.read_sql_query("SELECT * FROM team_game", nbadb).head(10)

,game_id,Team_Abbrev,H_A,win,fg,fga,fg3,fg3a,ft,fta,orb,drb,ast,stl,blk,tov,pf,pts
0,202110190LAL,GSW,A,1.0,41,93,14,39,25,30,9,41,30,9,2,17,18,121
1,202110190LAL,LAL,H,0.0,45,95,15,42,9,19,5,40,21,7,4,17,25,114
2,202110190MIL,BRK,A,0.0,37,84,17,32,13,23,5,39,19,3,9,12,17,104
3,202110190MIL,MIL,H,1.0,48,105,17,45,14,18,13,41,25,8,9,7,19,127
4,202110200CHO,CHO,H,1.0,46,107,13,31,18,27,12,34,29,9,5,8,21,123
5,202110200CHO,IND,A,0.0,42,90,17,47,21,24,8,43,29,2,10,16,24,122
6,202110200DET,CHI,A,1.0,37,86,7,23,13,15,9,39,18,8,5,17,19,94
7,202110200DET,DET,H,0.0,36,90,6,28,10,13,11,36,17,7,5,16,16,88
8,202110200MEM,CLE,A,0.0,47,93,14,38,13,18,7,29,38,6,5,10,17,121
9,202110200MEM,MEM,H,1.0,53,100,14,33,12,12,13,40,28,8,8,10,15,132
